In [ ]:
from datetime import datetime,timedelta
start=datetime(2020,12,30)
end=datetime(2022,1,1)

import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import pyspark.sql.functions as F
# ",
# tablemap={"arkham_stream_etl":"arkham","credinex_account_stream_etl":"credinex_account",
#           "credinex_hive_stream_etl":"credinex_hive","credinex_repeater_stream_etl":"credinex_repeater","protoss_stream_etl":"protoss"} #,
tablemap={
    "banda_stream_etl":"banda",
#     "lovina_stream_etl":"lovina",
#     "coupon_stream_etl":"coupon",
#     "notification_stream_etl":"notification",
#     "telemarket_stream_etl":"telemarket",
#     "arkham_stream_etl":"arkham",
#     "credinex_account_stream_etl":"credinex_account",
#     "credinex_hive_stream_etl":"credinex_hive",
#     "credinex_repeater_stream_etl":"credinex_repeater",
#     "protoss_stream_etl":"protoss"
} #,
databases=spark.sql("show databases")
dateList=[]
def gen_dates(b_date, days):
    day = timedelta(days=1)
    for i in range(days):
        yield b_date + day*i
def getDateList(start=None, end=None):
    """
    获取日期列表
    :param start: 开始日期
    :param end: 结束日期
    :return:
    """
    if start is None:
        start = datetime.strptime("2000-01-01", "%Y-%m-%d")
    if end is None:
        end = datetime.now()
    data = []
#     partitionsql = "ALTER TABLE `" + database+ "`." + tableNm+ "  ADD IF NOT EXISTS "
    for d in gen_dates(start, (end-start).days):
        data.append(d.strftime('%Y-%m-%d'))
#         datelist=d.strftime('%Y-%m-%d').split("-")
#         partition=" PARTITION (year=" + datelist[0] + ",month=" + datelist[1] + ",day=" + datelist[2] + ") "
#         partitionsql=partitionsql+partition
#         spark.sql(partitionsql)
    return data
dateList=getDateList(start,end)
databases=databases.collect()
def getpartitionSql(database,tableNm):
    partitionsql = "ALTER TABLE `" + database+ "`." + tableNm+ "  ADD IF NOT EXISTS "
    for date in dateList:
        datels=date.split("-")
        partition=" PARTITION (year=" + datels[0] + ",month=" + datels[1] + ",day=" + datels[2] + ") "
        partitionsql=partitionsql+partition
    return partitionsql
spark = SparkSession \
    .builder \
    .appName("Python Demo") \
    .config("hive.metastore.client.factory.class",
            "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
    .config("spark.driver.maxResultSize", "4g") \
    .enableHiveSupport() \
    .getOrCreate()
spark.conf.set("hive.exec.dynamic.partition","true");
spark.conf.set("hive.exec.dynamic.partition.mode","nonstrict");
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
for databaseName in tablemap:
    databasesql="show tables in "+databaseName
    tables=spark.sql(databasesql)
    tablelist=tables.collect();
    for row in tablelist:
        tableName=row["tableName"]
        if tableName=='t_password_update_record':
            print(tableName)
    #             tableName=row["tableName"]
            sql=getpartitionSql(databaseName,tableName)
    #         print(sql)
            spark.sql(sql)
    #             setPartitions(start,end,"partitions_etl",tableName)
    #                 spark.catalog.refreshTable(databaseName+"."+tableName)
    #                 sql1="SELECT * FROM "+databaseName+"."+tableName
    #                 path="s3://rupiahplus-data-warehouse/stream/demo/"+tableName;
    #                 spark.sql(sql1).write.mode("overwrite").partitionBy("year","month","day").orc(path)